In [4]:
!pip install tensorflow


  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB 1.2 MB/s eta 0:00:00
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/385.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/385.0 MB 3.5 MB/s eta 0:01:50


In [19]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk
import torch
import torch.nn.functional as F
from torchvision import models, transforms

# Define global variables
model = None
class_names = ['Jennie', 'Jisoo', 'Lisa', 'Rose']
CONFIDENCE_THRESHOLD = 0.7  # Set the confidence threshold to 70%

def load_model():
    global model
    file_path = filedialog.askopenfilename(title="Select model.pth file", filetypes=[("PyTorch Model", "*.pth")])
    
    if file_path:
        model = models.resnet50(weights=None)
        num_classes = len(class_names)
        model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
        model.load_state_dict(torch.load(file_path))
        model.eval()
        messagebox.showinfo("Success", "Model loaded successfully!")
        load_button.config(state=tk.DISABLED)
        predict_button.config(state=tk.NORMAL)
    else:
        messagebox.showerror("Error", "No model selected!")

def predict():
    if model:
        try:
            img_path = filedialog.askopenfilename(title="Select an image", filetypes=[("Image files", "*.jpg *.jpeg *.png")])
            
            if img_path:
                img = Image.open(img_path)
                img.thumbnail((300, 300))
                img_tk = ImageTk.PhotoImage(img)
                
                image_label.config(image=img_tk)
                image_label.image = img_tk
                
                input_tensor = preprocess(img)
                input_batch = input_tensor.unsqueeze(0)
                
                with torch.no_grad():
                    output = model(input_batch)
                
                # Apply softmax to get probabilities
                probabilities = F.softmax(output, dim=1)
                
                # Get the highest probability and its index
                max_prob, prediction_idx = torch.max(probabilities, dim=1)
                
                prediction_label = class_names[prediction_idx.item()]
                confidence = max_prob.item()
                
                if confidence >= CONFIDENCE_THRESHOLD:
                    result_text = f"Predicted: {prediction_label}\nConfidence: {confidence:.2%}"
                    result_label.config(text=result_text, fg="green")
                else:
                    result_text = f"Most likely not a member of BLACKPINK\nHighest confidence: {confidence:.2%} ({prediction_label})"
                    result_label.config(text=result_text, fg="red")
                
                progress_bar['value'] = 100
            else:
                messagebox.showerror("Error", "No image selected!")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")
    else:
        messagebox.showerror("Error", "No model loaded!")

# Tkinter window setup
root = tk.Tk()
root.title("BLACKPINK Member Classifier")
root.geometry("500x650")  # Increased height to accommodate larger result label
root.configure(bg='#F0F0F0')

style = ttk.Style()
style.theme_use('clam')

# Header
header_frame = tk.Frame(root, bg='#FF69B4')
header_frame.pack(fill=tk.X, pady=10)

header_label = tk.Label(header_frame, text="Facial Classfier", font=("Arial", 20, "bold"), bg='#FF69B4', fg='white')
header_label.pack(pady=10)

# Main content frame
content_frame = tk.Frame(root, bg='#F0F0F0')
content_frame.pack(padx=20, pady=10, fill=tk.BOTH, expand=True)

# Load Model Button
load_button = ttk.Button(content_frame, text="Load Model", command=load_model, style='TButton')
load_button.pack(pady=10)

# Image display
image_frame = tk.Frame(content_frame, bg='white', width=300, height=300)
image_frame.pack(pady=10)
image_frame.pack_propagate(0)

image_label = tk.Label(image_frame, bg='white')
image_label.pack(expand=True, fill=tk.BOTH)

# Predict Button
predict_button = ttk.Button(content_frame, text="Select Image and Predict", command=predict, state=tk.DISABLED)
predict_button.pack(pady=10)

# Progress Bar
progress_bar = ttk.Progressbar(content_frame, orient=tk.HORIZONTAL, length=300, mode='determinate')
progress_bar.pack(pady=10)

# Result Label
result_label = tk.Label(content_frame, text="", font=("Arial", 14), bg='#F0F0F0', wraplength=400)
result_label.pack(pady=10)

# Preprocessing setup
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

root.mainloop()

C:\Users\Srone\AppData\Local\Temp\ipykernel_23524\3693627510.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(file_path))
